In [171]:
class Node:
    ''' Represents a Node in AST'''
    
    def __init__(self):
        Node.indentStep = 4      # num spaces for each indentation level
        self.depth = 0           # level of the node in tree
        self.children = []       # Child nodes for this node
        
    def addChild(self, node):
        ''' Adds the node to childern list and sets the appropriate depth of child 
        node. Returns the curr node for chaining'''
        assert isinstance (node, Node)
        self.children.append(node)
        node.depth = self.depth + 1 
        return self
        
    def _indentSpace(self):
        ''' Returns the a string of spaces for indentation'''
        spaces = ' ' * (self.depth * Node.indentStep)
        return spaces
        
    def emitEntry(self):
        ''' Returns the gen code for entry to this block as string '''
        return self._indentSpace() + '{' + '\n'
    
    def emitExit(self):
        ''' Returns the gen code for exit from this block as string '''
        return self._indentSpace() + '}' + '\n'
    
    def emitDepthFirstOrder(self):
        ''' Visits the AST tree with this node as root return the code
        generated by depth first order traversal'''
        code = ''
        for child in self.children:
            code += child.emitEntry()
            code += child.emitDepthFirstOrder()
            code += child.emitExit()
        return code

In [172]:
nodes = Node()

In [173]:
nodes.addChild(Node()).addChild(Node())
nodes.children[1].addChild(Node()).addChild(Node())
nodes.children[1].children[1].addChild(Node())

In [174]:
print (nodes.emitDepthFirstOrder())

    {
    }
    {
        {
        }
        {
            {
            }
        }
    }



In [251]:
from string import Template
class ConvNode(Node):
    ''' Implements a Convolution Node'''
    def __init__(self,
                ifmAddr,        # int32_t Input Feature Map SRAM Address
                ifmWidth,        # int32_t Input Feature Map Width in bytes
                ifmHeight,       # int32_t Input Feature Map Height in bytes
                ifmDepth,        # int32_t Input Feature Map Depth in bytes
                kernelAddr,      # int32_t Kernel SRAM Address
                kernelBMAddr,    # int32_t Kernel Bit Mask Address
                kernelWidth,     # int32_t Kernel Width in bytes
                kernelHeight,    # int32_t Kernel Height in bytes
                kernelDepth,     # int32_t Kernel Depth in bytes
                biasAddr,        # int32_t Address of Bias
                convPadWidth,    # int32_t[4] Padding width [left, top, right, bottom]
                currLayerStride, # int32_t Stride in current layer
                nextLayerStride, # int32_t Stride in next layer
                ofmAddr,         # int32_t Output Feature Map SRAM Address
                ofmWidth,        # int32_t Output Feature Map Width in bytes
                ofmHeight,       # int32_t Output Feature Map Height in bytes
                ofmDepth):       # int32_t Output Feature Map Depth in bytes
        Node.__init__(self)
        
        entryTemplate = \
"""Conv(
        {0}, # int32_t Input Feature Map SRAM Address
        {1}, # int32_t Input Feature Map Width in bytes   
        {2}, # int32_t Input Feature Map Height in bytes
        {3}, # int32_t Input Feature Map Depth in bytes
        {4}, # int32_t Kernel SRAM Address
        {5}, # int32_t Kernel Bit Mask Address
        {6}, # int32_t Kernel Width in bytes
        {7}, # int32_t Kernel Height in bytes
        {8}, # int32_t Kernel Depth in bytes
        {9}, # int32_t Address of Bias
        {10}, # int32_t[4] Padding width [left, top, right, bottom]
        {11}, # int32_t Stride in current layer
        {12}, # int32_t Stride in next layer
        {13}, # int32_t Output Feature Map SRAM Address
        {14}, # int32_t Output Feature Map Width in bytes
        {15}, # int32_t Output Feature Map Height in bytes
        {16}  # int32_t Output Feature Map Depth in bytes
       ).execute()\n"""

        self.entryText = entryTemplate.format (
                                                ifmAddr         ,
                                                ifmWidth        ,
                                                ifmHeight       ,
                                                ifmDepth        ,
                                                kernelAddr      ,
                                                kernelBMAddr    ,
                                                kernelWidth     ,
                                                kernelHeight    ,
                                                kernelDepth     ,
                                                biasAddr        ,
                                                convPadWidth    ,
                                                currLayerStride ,
                                                nextLayerStride ,
                                                ofmAddr         ,
                                                ofmWidth        ,
                                                ofmHeight       ,
                                                ofmDepth
                                              )
        self.emitText = ''
        
    def emitEntry(self): 
        ''' Emits the Convolution operation'''
        return self._indentSpace() + self.entryText
        
    def emitExit(self):
        ''' Emits an empty string. No epilogue needed'''
        return self.emitText



In [252]:
node = ConvNode(
                str( 0),
                str( 1),
                str( 2),
                str( 3),
                str( 4),
                str( 5),
                str( 6),
                str( 7),
                str( 8),
                str( 9),
                str(10),
                str('{11,11,11,11}'),
                str(12),
                str(13),
                str(14),
                str(15),
                str(16),
               )

In [253]:
nodes = Node()

In [254]:
nodes.addChild(Node()).addChild(node)

In [255]:
print (nodes.children)

[<__main__.Node object at 0x7efbdde06320>, <__main__.ConvNode object at 0x7efbdded84e0>]


In [256]:
print (nodes.emitDepthFirstOrder())

    {
    }
    Conv(
        0, # int32_t Input Feature Map SRAM Address
        1, # int32_t Input Feature Map Width in bytes   
        2, # int32_t Input Feature Map Height in bytes
        3, # int32_t Input Feature Map Depth in bytes
        4, # int32_t Kernel SRAM Address
        5, # int32_t Kernel Bit Mask Address
        6, # int32_t Kernel Width in bytes
        7, # int32_t Kernel Height in bytes
        8, # int32_t Kernel Depth in bytes
        9, # int32_t Address of Bias
        10, # int32_t[4] Padding width [left, top, right, bottom]
        {11,11,11,11}, # int32_t Stride in current layer
        12, # int32_t Stride in next layer
        13, # int32_t Output Feature Map SRAM Address
        14, # int32_t Output Feature Map Width in bytes
        15, # int32_t Output Feature Map Height in bytes
        16  # int32_t Output Feature Map Depth in bytes
       ).execute()



In [257]:
class LoopNode(Node):
    ''' Implements a for-Loop Node'''
    def __init__(self,
                 counterName,# Name of counter variable
                 initVal,    # intialialize for loop counter
                 boundVal,   # bound for loop (excluding the bound Val)
                 incVal      # step of increment for loop counter
                ):
        Node.__init__(self)
        
        entryTemplate = 'for ({0} = {1}; {0} < {2}; {0} += {3}) {{\n'

        self.entryText = entryTemplate.format (counterName, initVal, boundVal, incVal) 
        self.emitText = '}\n'
        
    def emitEntry(self): 
        ''' Emits the for loop operation'''
        return self._indentSpace() + self.entryText
        
    def emitExit(self):
        ''' Emits the closing braces'''
        return self._indentSpace() + self.emitText



In [258]:
loop = LoopNode('i',0,10,1)

In [259]:
nodes = Node()
nodes.addChild(Node()).addChild(loop)
nodes.children[1].addChild(node)

In [260]:
print (nodes.emitDepthFirstOrder())

    {
    }
    for (i = 0; i < 10; i += 1) {
        Conv(
        0, # int32_t Input Feature Map SRAM Address
        1, # int32_t Input Feature Map Width in bytes   
        2, # int32_t Input Feature Map Height in bytes
        3, # int32_t Input Feature Map Depth in bytes
        4, # int32_t Kernel SRAM Address
        5, # int32_t Kernel Bit Mask Address
        6, # int32_t Kernel Width in bytes
        7, # int32_t Kernel Height in bytes
        8, # int32_t Kernel Depth in bytes
        9, # int32_t Address of Bias
        10, # int32_t[4] Padding width [left, top, right, bottom]
        {11,11,11,11}, # int32_t Stride in current layer
        12, # int32_t Stride in next layer
        13, # int32_t Output Feature Map SRAM Address
        14, # int32_t Output Feature Map Width in bytes
        15, # int32_t Output Feature Map Height in bytes
        16  # int32_t Output Feature Map Depth in bytes
       ).execute()
    }

